### In this notebook I will run the hypertuning for what regards the models trained with pseudolikelihood. To do so we will leverage the package Optuna. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys


sys.path.insert(1, "./../util")
sys.path.insert(1, "./../model")
from encoded_protein_dataset_new import get_embedding, EncodedProteinDataset_new, collate_fn_new#, dynamic_collate_fn
from pseudolikelihood import get_npll2, get_npll_indep
import torch, torchvision
from torch.nn.utils import clip_grad_norm_
from potts_decoder import PottsDecoder
from torch.utils.data import DataLoader, RandomSampler
from functools import partial
import biotite.structure
from biotite.structure.io import pdbx, pdb
from biotite.structure.residues import get_residues
from biotite.structure import filter_backbone
from biotite.structure import get_chains
from biotite.sequence import ProteinSequence
from typing import Sequence, Tuple, List
import scipy
from tqdm import tqdm
import pandas as pd
import csv
import time
from torch.utils.tensorboard import SummaryWriter

##TURIN HPC
#sys.path.insert(1, "/Data/silva/esm/")

## EUROPA
#sys.path.insert(1, "/home/lucasilva/esm/")

##Lucas computer
sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/esm/")
import esm
#from esm.inverse_folding import util
import esm.pretrained as pretrained
from ioutils import read_fasta, read_encodings
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict
from Bio import SeqIO

from dynamic_loader import dynamic_collate_fn, dynamic_cluster
import optuna



In [2]:
### IDEA: MSAS PROCEDURE CAN GIVE DIFFERENT OUTPUT SHAPES? ASK
max_msas = None
#msa_dir = "/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/msas/"
msa_dir = "/home/luchinoprince/split2/"
encoding_dir ="/media/luchinoce/split2/"
encoding_dir ="/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/"

train_dataset = EncodedProteinDataset_new(os.path.join(msa_dir, 'train'), encoding_dir, noise=0.02, max_msas=max_msas)          ## Default value of noise used
sequence_test_dataset = EncodedProteinDataset_new(os.path.join(msa_dir, 'test/sequence'), encoding_dir, noise=0.0, max_msas=max_msas)
structure_test_dataset = EncodedProteinDataset_new(os.path.join(msa_dir, 'test/structure'), encoding_dir, noise=0.0, max_msas=max_msas)
superfamily_test_dataset = EncodedProteinDataset_new(os.path.join(msa_dir, 'test/superfamily'), encoding_dir, noise=0.0, max_msas=max_msas)
print(f"I have loaded the train and test datasets: train:{len(train_dataset)}, seq:{len(sequence_test_dataset)}, struc:{len(structure_test_dataset)}, super:{len(superfamily_test_dataset)}")

/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/InverseFolding/train/./../util/encoded_protein_dataset_new.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings = torch.tensor(read_encodings(encoding_path, trim=False))


/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxh02.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/1vwxh02_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/3o58E00.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/3o58E00_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/2wc7A01.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/2wc7A01_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxP00.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/train/1vwxP00_train.a3m.pt
/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/structure_encodings/1vwxG01.encodings.pt Mismatch in dimension, skipping /home/luchinoprince/split2/tr

### I DON'T CREATE THE DATALOADERS AS THEY WILL DEPEND ON THE HYPERPARAMETERS

In [3]:
decoder = None
embedding = None
torch.cuda.empty_cache()

q=21

seed = 24877
torch.random.manual_seed(seed)
np.random.seed(seed)


bk_iter = False                                                  ## This tells us how ofter we save a model(default values is every ten-thousand updates)
#n_epochs = update_steps//(len(train_dataset)//batch_structure_size)   ## the other update steps will be used for "partial epochs", I want to save the last complet epoch
#print(f"With update_steps:{update_steps} we will do {n_epochs} full epochs")

input_encoding_dim = 512
param_embed_dim = 512
n_param_heads = 48
d_model = 512 ##old 512
n_heads = 8 ## old 8
n_layers = 6
## Check before running which is the GPU which is free the most and put it as the running device
device = 0        ## DON'T SET TO ONE OTHER THAN IN SPECIAL SPECIAL OCCASIONS, VERY NOISYYYYY!
dropout = 0.1

decoder = PottsDecoder(q, n_layers, d_model, input_encoding_dim, param_embed_dim, n_heads, n_param_heads, dropout=dropout)
decoder.to(device)
embedding = get_embedding(q)
embedding.to(device)

Embedding(22, 21)

In [4]:
def get_loss_new(decoder, inputs, eta_J, eta_h):
    """eta_J is the multiplicative term in front of the penalized negative pseudo-log-likelihood for the Couplings
        if no penalty is proveded for the fields, we put the equal to one another """
    msas, encodings, padding_mask  = [input.to(device, non_blocking=True) for input in inputs]
    B, M, N = msas.shape
    #print(f"encodings' shape{encodings.shape}, padding mask:{padding_mask.shape}")
    param_embeddings, fields = decoder.forward_new(encodings, padding_mask)
    msas_embedded = embedding(msas)
    #eta = eta_J
    # get npll
    npll = get_npll2(msas_embedded, param_embeddings, fields, N, q)
    padding_mask_inv = (~padding_mask)
    # multiply with the padding mask to filter non-existing residues (this is probably not necessary)       
    npll = npll * padding_mask_inv.unsqueeze(1)
    npll_mean = torch.sum(npll) / (M * torch.sum(padding_mask_inv))
    
    Q = torch.einsum('bkuia, buhia->bkhia', 
                param_embeddings.unsqueeze(2), param_embeddings.unsqueeze(1)).sum(axis=-1)
    #penalty = eta*(torch.sum(torch.sum(Q,axis=-1)**2) - torch.sum(Q**2) + torch.sum(fields**2))/B
    penalty = eta_J/B*(torch.sum(torch.sum(Q,axis=-1)**2) - torch.sum(Q**2)) + eta_h/B*torch.sum(fields**2) 
    loss_penalty = npll_mean + penalty
    return loss_penalty, npll_mean.item() 

def get_loss_indep(decoder, inputs, eta):
    """eta is the multiplicative term in front of the penalized negative pseudo-log-likelihood"""
    msas, encodings, padding_mask  = [input.to(device, non_blocking=True) for input in inputs]
    B, M, N = msas.shape
    #print(f"encodings' shape{encodings.shape}, padding mask:{padding_mask.shape}")
    fields = decoder.forward_indep(encodings, padding_mask)
    msas_embedded = embedding(msas)

    # get npll
    npll = get_npll_indep(msas_embedded, fields, N, q)
    padding_mask_inv = (~padding_mask)
    # multiply with the padding mask to filter non-existing residues (this is probably not necessary)       
    npll = npll * padding_mask_inv.unsqueeze(1)
    npll_mean = torch.sum(npll) / (M * torch.sum(padding_mask_inv))
    
    #Q = torch.einsum('bkuia, buhia->bkhia', 
    #            param_embeddings.unsqueeze(2), param_embeddings.unsqueeze(1)).sum(axis=-1)
    penalty = eta*(torch.sum(fields**2))/B
    loss_penalty = npll_mean + penalty
    return loss_penalty, npll_mean.item() 


def get_loss_loader(decoder, loader, eta_J, eta_h):
    decoder.eval()
    losses = []
    #with torch.no_grad():
    for effective_batch_size, inputs_packed in loader:
        npll_full = 0
        for inputs in inputs_packed:
            mini_batch_size = inputs[0].shape[0]
            #_, npll = get_loss_indep(decoder, inputs, eta_J, eta_h) ## For independent model without couplings
            _, npll = get_loss_new(decoder, inputs, eta_J, eta_h)
            npll_full += npll*mini_batch_size/effective_batch_size
        losses.append(npll_full)
            #del inputs
    
    return np.mean(losses)

def train(decoder, inputs_packed, eta_J, eta_h, optimizer, scaler):
    effective_batch_size = inputs_packed[0]
    loss_penalty_full = 0
    train_loss_full = 0
    optimizer.zero_grad(set_to_none=True)                           ## set previous gradients to 0
    with torch.cuda.amp.autocast():  ## autocasting mixed precision
        for inputs in inputs_packed[1]:
            mini_batch_size = inputs[0].shape[0]
            #loss_penalty, train_batch_loss = get_loss_indep(decoder, inputs, eta_J, eta_h)    ## get the current loss for the batch this is for the independent training
            loss_penalty, train_batch_loss = get_loss_new(decoder, inputs, eta_J, eta_h)
            loss_penalty = loss_penalty * mini_batch_size/effective_batch_size
            train_batch_loss = train_batch_loss * mini_batch_size/effective_batch_size
            #loss_penalty.backward()                         ## Get gradients
            scaler.scale(loss_penalty).backward()
            loss_penalty_full += loss_penalty.detach()
            train_loss_full += train_batch_loss
    
    
    scaler.step(optimizer)
    scaler.update()
    #optimizer.step()   

    return loss_penalty_full, train_loss_full

In [6]:
def train_and_evaluate(param, decoder, trial):
    batch_msa_size = param['batch_msa_size']
    batch_structure_size = 32   ### I think with empty GPU we can go up to 16 easily
    perc_subset_test = 1.0     ## During the training, for every dataset available we select a random 10% of its samples
    q = 21 ##isn't always 21
    
    ############################### CREATE DATALOADERS ###############################################################
    collate_fn = partial(dynamic_collate_fn, q=q, batch_size=batch_structure_size, batch_msa_size=batch_msa_size)
    train_loader = DataLoader(train_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=True,
    num_workers=4, pin_memory=True)


    sequence_test_loader = DataLoader(sequence_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
    num_workers=4, pin_memory=True, sampler=RandomSampler(sequence_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(sequence_test_dataset))))

    structure_test_loader = DataLoader(structure_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
    num_workers=4, pin_memory=True, sampler=RandomSampler(structure_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(structure_test_dataset))))

    superfamily_test_loader = DataLoader(superfamily_test_dataset, batch_size=batch_structure_size, collate_fn=collate_fn, shuffle=False, 
    num_workers=4, pin_memory=True, sampler=RandomSampler(superfamily_test_dataset, replacement=True, num_samples=int(perc_subset_test*len(superfamily_test_dataset))))

    optimizer = torch.optim.AdamW(decoder.parameters(), lr=param['lr'])
    eta_J = param['eta_J']
    eta_h = param['eta_h']
    epoch = 0.0
    train_batch_losses = []
    update_step = 0
    scaler = torch.cuda.amp.GradScaler()
    update_steps = 70000
    ## I run 100 epochs
    #while epoch < 100:
    while update_step < update_steps:
        for inputs_packed in train_loader:
            update_step += 1                                ## Increase update step (the update steps will count also different batches within the same epoch)
            print(f"I am at epoch: {update_step}", end="\r")
            decoder.train()
            loss_penalty, train_batch_loss = train(decoder, inputs_packed, eta_J, eta_h, optimizer, scaler)
            epoch = update_step / len(train_loader)
            loss_penalty.detach()
            train_batch_losses.append(train_batch_loss)
            if (update_step == 1) or (epoch % 5 == 0):
                train_loss = np.mean(train_batch_losses)
                with torch.no_grad():
                    sequence_test_loss = get_loss_loader(decoder, sequence_test_loader, eta_J, eta_h)
                    structure_test_loss = get_loss_loader(decoder, structure_test_loader, eta_J, eta_h)
                    superfamily_test_loss = get_loss_loader(decoder, superfamily_test_loader, eta_J, eta_h)
                #### REPORT TO ALLOW PRUNING, we wait at least 10 EPOCHS to prune ####
                if epoch > 6:
                    trial.report((sequence_test_loss + structure_test_loss + superfamily_test_loss/3), epoch)
                    if trial.should_prune():
                        raise optuna.exceptions.TrialPruned()
                
                train_batch_losses = []

    
    return sequence_test_loss, structure_test_loss, superfamily_test_loss


def objective(trial):
    param = {
            'lr' : trial.suggest_uniform('lr', 1e-5, 1e-3),
            'eta_J' : trial.suggest_uniform('eta_J', 1e-6, 1e-4),
            'eta_h' : trial.suggest_uniform('eta_h', 1e-5, 1e-3),
            'batch_msa_size' : trial.suggest_categorical('batch_msa_size', [16, 32, 64])
            }
    
    ## I Recreate otherwise I would risk to start the new training at the previous end.
    decoder = PottsDecoder(q, n_layers, d_model, input_encoding_dim, param_embed_dim, n_heads, n_param_heads, dropout=dropout)
    decoder.to(device)
    embedding = get_embedding(q)
    embedding.to(device)
    

    test1, test2, test3 = train_and_evaluate(param, decoder, trial)

    ### This to try to help memory consumption
    #decoder = None
    #torch.cuda.empty_cache()
    return (test1+test2+test3)/3


    


In [7]:
import warnings
warnings.filterwarnings("ignore")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20, gc_after_trial=True)

[I 2023-05-05 19:09:26,001] A new study created in memory with name: no-name-14abe771-7671-48f1-90c1-6b6f321f79c2


[I 2023-05-05 22:48:55,463] Trial 0 finished with value: 2.6673463899704113 and parameters: {'lr': 0.0005648914550632188, 'eta_J': 6.895952006905523e-05, 'eta_h': 0.0003256236974166933, 'batch_msa_size': 32}. Best is trial 0 with value: 2.6673463899704113.


[I 2023-05-06 03:35:15,978] Trial 1 finished with value: 14.512913838442719 and parameters: {'lr': 0.000931126492364323, 'eta_J': 5.1522358244928086e-05, 'eta_h': 0.00032270953769181736, 'batch_msa_size': 64}. Best is trial 0 with value: 2.6673463899704113.


[W 2023-05-06 06:35:16,066] Trial 2 failed with parameters: {'lr': 0.0009026428828152847, 'eta_J': 8.891034988130351e-05, 'eta_h': 0.00018547525344703595, 'batch_msa_size': 16} because of the following error: The value nan is not acceptable..
[W 2023-05-06 06:35:16,067] Trial 2 failed with value nan.


[I 2023-05-06 11:22:01,979] Trial 3 finished with value: 1.7372174856570075 and parameters: {'lr': 0.00019445081758246309, 'eta_J': 7.786339549555277e-05, 'eta_h': 0.0006977184495610499, 'batch_msa_size': 64}. Best is trial 3 with value: 1.7372174856570075.


...

[I 2023-05-07 13:23:47,961] Trial 15 pruned. 


[I 2023-05-07 18:10:43,149] Trial 16 finished with value: 1.6943064924131714 and parameters: {'lr': 0.00023704241710699475, 'eta_J': 2.493167062989222e-06, 'eta_h': 0.0006453437955003415, 'batch_msa_size': 64}. Best is trial 14 with value: 1.6797838518972414.


[I 2023-05-07 18:41:00,707] Trial 17 pruned. 


[I 2023-05-07 22:20:56,541] Trial 18 finished with value: 1.6995136189893474 and parameters: {'lr': 0.0001920510375765753, 'eta_J': 1.7430346570350896e-05, 'eta_h': 0.0004822371832505785, 'batch_msa_size': 32}. Best is trial 14 with value: 1.6797838518972414.


[I 2023-05-07 22:51:10,949] Trial 19 pruned. 


In [8]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

lr: 0.0003624665966690978
eta_J: 2.1486061245002986e-06
eta_h: 0.0006424942117327073
batch_msa_size: 64
